In [21]:
from loader import *
from fastNLP.io import ChnSentiCorpLoader
from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import ChnSentiCorpPipe, DataBundle
from spacy.lang.zh import Chinese
import spacy
from fastNLP.embeddings import StaticEmbedding, StackEmbedding
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import KFold


nlp = Chinese()
nlp = Chinese.from_config({"nlp": {"tokenizer":{"segmenter": "pkuseg"}}})
nlp.tokenizer.initialize(pkuseg_model="mixed", pkuseg_user_dict='./data/dict.txt')

In [22]:
from fastNLP.io.pipe import CWSPipe, ChnSentiCorpPipe
pipe =ChnSentiCorpPipe()


In [ ]:
# risk_raw = './data/Train_risk_classification_ans_speaker_illness.csv'
# risk_df = pd.read_csv(risk_raw, index_col=0)
# ds =


In [23]:
risk_raw = './data/Train_risk_classification_ans_speaker_illness.csv'
risk_df = pd.read_csv(risk_raw, index_col=0)
risk_df

ds = {}
for i, (train_idx, val_idx) in enumerate(KFold(n_splits=5).split((risk_df))):
    train_df = risk_df.iloc[train_idx]
    val_df = risk_df.iloc[val_idx]

    ds.update({
        f'train{i}': DataSet({'raw_words':train_df['illness'].tolist(), 'target':train_df['label'].astype(int).tolist()}),
        f'val{i}':   DataSet({'raw_words':val_df['illness'].tolist(), 'target':val_df['label'].astype(int).tolist()}),
    })

for d in ds.values():
    # d.apply(lambda ins: [ele.text for ele in nlp.tokenizer.pipe(ins['raw_words'])], new_field_name='words', is_input=True)
    # d.add_seq_len(field_name='words')
    # d.set_input('target', 'seq_len')   
    d.set_target('target')

In [24]:
bundle = DataBundle(None, ds )
pipe.process(bundle)

Exception happens at the `0`th instance.
When processing the `0` dataset, the following error occurred:


AssertionError: no such field:chars in Instance +-------------------------------------------+--------+
| raw_words                                 | target |
+-------------------------------------------+--------+
| 個管師：嗯嗯。民眾：然後我就是在學校做... | 0      |
+-------------------------------------------+--------+

In [19]:
bundle.get_dataset('train1')

+-------------------------+------------------------+------------------------+---------+
| raw_words               | target                 | chars                  | seq_len |
+-------------------------+------------------------+------------------------+---------+
| 個管師：12月那時候？... | [1, 0, 0, 0, 0, 0, ... | [11, 42, 5, 2, 53, ... | 910     |
| 民眾：沒有。個管師：... | [1, 0, 0, 0, 0, 0, ... | [7, 8, 2, 25, 10, 3... | 1013    |
| 民眾：啊就在忙裝潢啊... | [1, 0, 0, 0, 0, 0, ... | [7, 8, 2, 31, 13, 4... | 895     |
| 民眾：對啊。個管師：... | [1, 0, 0, 0, 0, 0, ... | [7, 8, 2, 24, 31, 3... | 1022    |
| 個管師：好喔，過去這... | [1, 0, 0, 0, 0, 0, ... | [11, 42, 5, 2, 15, ... | 728     |
| 民眾：好。個管師：那... | [1, 0, 0, 0, 0, 0, ... | [7, 8, 2, 15, 3, 11... | 984     |
| 個管師：我剛剛大概有... | [1, 0, 0, 0, 0, 0, ... | [11, 42, 5, 2, 12, ... | 916     |
| 個管師：剛剛王醫師有... | [1, 0, 0, 0, 0, 0, ... | [11, 42, 5, 2, 141,... | 919     |
| 個管師：誒，我有丟那... | [1, 0, 0, 0, 0, 0, ... | [11, 42, 5, 2, 263,... | 874     |
| 民眾：持續。個管師：... | [1, 0, 0, 0, 0,

In [9]:
label_ds = DataSet({'target':[0, 1]})
label_vocab  = Vocabulary(unknown=None, padding=None).from_dataset(label_ds, field_name='target')

char_vocab = Vocabulary()
char_vocab.from_dataset(*ds.values(), field_name='words'
# , no_create_entry_dataset=[ds['val']]
)
char_vocab.index_dataset(*ds.values(), field_name='words')

label_vocab.from_dataset(label_ds, field_name='target')
label_vocab.index_dataset(label_ds, field_name='target')



Exception happens at the `0`th instance.
When processing the `0` dataset, the following error occurred:


AssertionError: no such field:words in Instance +-------------------------------------------+--------+
| raw_words                                 | target |
+-------------------------------------------+--------+
| 個管師：嗯嗯。民眾：然後我就是在學校做... | 0      |
+-------------------------------------------+--------+

In [5]:

bundle = DataBundle({ 'target':label_vocab}, ds )

In [6]:
bundle

In total 10 datasets:
	train0 has 276 instances.
	val0 has 70 instances.
	train1 has 277 instances.
	val1 has 69 instances.
	train2 has 277 instances.
	val2 has 69 instances.
	train3 has 277 instances.
	val3 has 69 instances.
	train4 has 277 instances.
	val4 has 69 instances.
In total 1 vocabs:
	target has 2 entries.

In [7]:
from torch import nn
from fastNLP.modules import LSTM
import torch
from fastNLP.models.cnn_text_classification import CNNText


class BiLSTMMaxPoolCls(nn.Module):
    def __init__(self, embed, hidden_size=300, num_layers=3, dropout=0.3):
        super().__init__()
        self.embed = embed

        self.lstm = LSTM(self.embed.embedding_dim, hidden_size=hidden_size//2, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 2)


    def forward(self, words, seq_len):  # 这里的名称必须和DataSet中相应的field对应，比如之前我们DataSet中有chars，这里就必须为chars
        # chars:[batch_size, max_len]
        # seq_len: [batch_size, ]
        chars = self.embed(words)
        # print(chars.shape)
        outputs, _ = self.lstm(chars, seq_len.cpu())
        outputs = self.dropout_layer(outputs)
        outputs, _ = torch.max(outputs, dim=1)
        outputs = self.fc(outputs)

        return {'pred':outputs}  # [batch_size,], 返回值必须是dict类型，且预测值的key建议设为pred



In [8]:
bundle.get_dataset(f'train{0}')

+-------------------------+--------+------------------------+---------+
| raw_words               | target | words                  | seq_len |
+-------------------------+--------+------------------------+---------+
| 個管師：嗯嗯。民眾：... | 0      | [11, 42, 5, 2, 35, ... | 930     |
| 個管師：過去這一個月... | 1      | [11, 42, 5, 2, 85, ... | 929     |
| 個管師：這個月還好嗎... | 1      | [11, 42, 5, 2, 17, ... | 838     |
| 個管師：喔！好喔。你... | 1      | [11, 42, 5, 2, 34, ... | 995     |
| 個管師：今天超多人誒... | 1      | [11, 42, 5, 2, 121,... | 2077    |
| 個管師：所以這三個月... | 1      | [11, 42, 5, 2, 53, ... | 950     |
| 個管師：哈哈哈。民眾... | 1      | [11, 42, 5, 2, 93, ... | 973     |
| 個管師：好喔，這個月... | 1      | [11, 42, 5, 2, 15, ... | 871     |
| 民眾：沒有關係是吧？... | 1      | [7, 8, 2, 25, 10, 1... | 1096    |
| 民眾：三個禮拜以內…...  | 0      | [7, 8, 2, 99, 11, 9... | 870     |
| 民眾：還好。個管師：... | 1      | [7, 8, 2, 33, 15, 3... | 933     |
| 個管師：對，所以其實... | 1      | [11, 42, 5, 2, 24, ... | 1030    |
| ...                     | ...    | ..

In [9]:
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam
from fastNLP import AccuracyMetric


loss = CrossEntropyLoss()
metric = AccuracyMetric()
device = 0 if torch.cuda.is_available() else 'cpu'

for idx in range(5):
  print(f'-----------Fold{idx}------------')

  word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name='cn-fasttext')
  embed = StackEmbedding([word2vec_embed])
  model = CNNText(embed, 2, dropout=0.2)
  optimizer = Adam(model.parameters(), lr=0.001)

  trainer = Trainer(
    train_data=bundle.get_dataset(f'train{idx}'), 
    dev_data=bundle.get_dataset(f'val{idx}'),
    model=model, loss=loss, device=device,
    optimizer=optimizer, batch_size=16,
    metrics=metric,       
  )
  trainer.train()  


-----------Fold0------------
Found 2250 out of 2267 words in the pre-training embedding.
input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 930]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2021-06-13-21-40-56-881589


  0%|          | 0/180 [00:00<?, ?it/s, loss:{0:<6.5f}]

  0%|          | 0/5 [00:00<?, ?it/s]

Evaluate data in 0.01 seconds!
Evaluation on dev at Epoch 1/10. Step:18/180: 
AccuracyMetric: acc=0.571429



  0%|          | 0/5 [00:00<?, ?it/s]

...

Evaluate data in 0.04 seconds!
Evaluation on dev at Epoch 8/10. Step:144/180: 
AccuracyMetric: acc=0.782609



  0%|          | 0/5 [00:00<?, ?it/s]

Evaluate data in 0.04 seconds!
Evaluation on dev at Epoch 9/10. Step:162/180: 
AccuracyMetric: acc=0.826087



  0%|          | 0/5 [00:00<?, ?it/s]

Evaluate data in 0.04 seconds!
Evaluation on dev at Epoch 10/10. Step:180/180: 
AccuracyMetric: acc=0.826087

Reloaded the best model.

In Epoch:9/Step:162, got best dev performance:
AccuracyMetric: acc=0.826087


In [10]:

# 在测试集上测试一下模型的性能
from fastNLP import Tester
print("Performance on test is:")
# tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
# tester.test()


Performance on test is:
